----

# **Weaviate, LangChain, Cohere - RAG-based application to control biases in media articles**

----

In [ ]:
https://newsdata.io/api/1/archive?apikey=pub_376814b12a2e29cbabbcdbba21786022e96c8&q=

In [12]:
response = requests.get("https://newsdata.io/api/1/news?apikey=pub_376814b12a2e29cbabbcdbba21786022e96c8&q=Israel&language=en")
response_json = response.json()

In [14]:
#response_json

In [ ]:
#pytorch python 3.9
#!pip install weaviate-client

In [2]:
import requests
import pandas as pd
import numpy as np
import os
import json
import weaviate
from weaviate.embedded import EmbeddedOptions
from json import loads, dumps

## **1. News API - Get articles**

In [ ]:
#english only documents

In [15]:
## api key for News API
news_api_key = "b6d1e4ad43564755af9a28ec02ba47f4"

In [16]:
#define subjects
list_of_subjects = ["Israel", "Immigration", "Elections", "NFL", "United Nations", "Gaza", "Biden", "Trump", "Republicans", "Democrats"]

columns = ['subject', 'source_id', 'name', 'author', 'title', 'url', 'urlToImage', 'publishedAt', 'content']

#create empty dataframe to fill
df_original = pd.DataFrame(columns=columns)
    
#loop over subjects and get 100 requests

for subject in list_of_subjects:
    

    #define url
    url = (f"https://newsapi.org/v2/everything?"
               f"q={subject}&"
               f"language=en&"
               f"apiKey={news_api_key}")

    response = requests.get(url)
    response_json = response.json()
    
    
    list_of_response = []

    for response in response_json['articles']:

        #add subject
        subject = subject
        
        #get individual columns from data    
        source_id = response['source']['id']
        name = response['source']['name']
        author = response['author']
        title = response['title']
        url = response['url']
        urlToImage = response['urlToImage']
        publishedAt = response['publishedAt']
        content = response['content']

        #add to list
        list_of_response.append([subject, source_id, name, author, title, url, urlToImage, publishedAt, content])

    df_1 = pd.DataFrame(list_of_response, columns=columns)
        
    #add single df to multiple df
    df_original = df_original.append(df_1)

/tmp/ipykernel_4030/162420850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(df_1)
/tmp/ipykernel_4030/162420850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(df_1)
/tmp/ipykernel_4030/162420850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(df_1)
/tmp/ipykernel_4030/162420850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(df_1)
/tmp/ipykernel_4030/162420850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [17]:
df_original.head()

,subject,source_id,name,author,title,url,urlToImage,publishedAt,content
0,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,"Hamas command in north Gaza destroyed, Israel ...",https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-06T20:36:02Z,"The Israeli army says it has ""completed the di..."
1,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,South Africa's genocide case against Israel: B...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-12T20:59:40Z,The UN's top legal body has now heard two days...
2,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,UN court to hear South Africa genocide case ag...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-11T07:16:13Z,The UN's International Court of Justice will b...
3,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,Khan Younis: Israel says forces have encircled...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-23T21:15:25Z,The Israeli military says its ground forces ha...
4,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,Israeli military says 21 soldiers killed in Gaza,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-23T07:00:02Z,The Israeli army says 21 of its soldiers have ...


In [ ]:
df_original.columns

In [ ]:
df_original.shape

In [ ]:
#save to excel
df_original.to_excel("news_2024_02_02.xlsx", index=False)

----------------------

-------

## **2. Weaviate, Cohere, LangChain application**

### **Create Weaviate client**

- Cohere embeddings models are used
- Weaviate as vector db
- LangChain

In [12]:
#cohere key received from hackathon
cohere_key = "INe0q0lAiANgPiD2DtehYAF85XftQQjJxXQkMi7h"

client = weaviate.Client(
  embedded_options=EmbeddedOptions(additional_env_vars={
        "ENABLE_MODULES":
        "text2vec-cohere, generative-cohere, reranker-cohere"}),    #this uses Cohere 3x: 1) For text 2 embeddings, 2) for Generative answer, 3) for Re-ranking
    
        additional_headers={"X-Cohere-Api-Key": cohere_key})

embedded weaviate is already listening on port 8079


In [ ]:
### NEW

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

import os
import json
# only if we want to import a prompt
# from langchain import hub
from langchain_community.vectorstores import Weaviate
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

from langchain_community.llms import Cohere
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.docstore.document import Document
import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain_community.embeddings import CohereEmbeddings
from json import loads, dumps

In [9]:
WV_API_KEY = "cZBd01Q88KEBdZcHKdJ2bsjIpYIsck3CDsX9"
COHERE_KEY = "INe0q0lAiANgPiD2DtehYAF85XftQQjJxXQkMi7h"


# To adapt to Weaviate this one must be used even for English
EMBED_MODEL = "embed-multilingual-v3.0"

auth_client_secret = weaviate.AuthApiKey(api_key=WV_API_KEY)

embed_model = CohereEmbeddings(model=EMBED_MODEL, cohere_api_key=COHERE_KEY)

client = weaviate.Client(
    url="https://orals-cluster1-j7b4goli.weaviate.network",
    auth_client_secret=auth_client_secret,
    additional_headers={"X-Cohere-Api-Key": COHERE_KEY}
)

/home/datascience/conda/pytorch20_p39_gpu_v2/lib/python3.9/site-packages/weaviate/__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
/home/datascience/conda/pytorch20_p39_gpu_v2/lib/python3.9/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


### **Create a new table/schema in the Weaviate db**

In [11]:
# We delete the class in case this template has already been run.
client.schema.delete_class("news_articles")

#new schema with definitions

{
  "classes": [
    {
      "class": "Document",
#       ...,
      "moduleConfig": {
        "reranker-cohere": {
            "model": "rerank-multilingual-v2.0",
        },
      }
    }
  ]
}


article_schema = {
    "class": "news_articles",
    "description": "News articles",
    "vectorizer": "text2vec-cohere",
    "moduleConfig": {                                                 ## Config for Text to Vecotr
        "text2vec-cohere": {
            "model": "multilingual-22-12",
            "truncate": "RIGHT"
        },
        "reranker-cohere": {                                           ## config for re-rank
            "model": "rerank-multilingual-v2.0",
        },        
        
        "generative-cohere": {                                        ## Config for GenAI 
            "model": "command-xlarge-nightly",
        }
    },
    "vectorIndexConfig": {
        "distance": "dot"
    },
    "properties": [
    {
        "name": "content_full",
        "dataType": [ "text" ],
        "description": "Article body",
        "moduleConfig": {
            "text2vec-cohere": {
                "skip": False,                                                     ## Only text to vector for content
                "vectorizePropertyName": False
            }
        }
    },
    {
        "name": "subject",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }                ### skip = true means not vectorizing. You can also do img2vec when there is an image.
    },
    {
        "name": "source_id",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "name",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "author",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "title",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "url",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "urlToImage",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "publishedAt",
        "dataType": [ "text" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    }
    ]
}

# add the schema
client.schema.create_class(article_schema)
print("The collection schema has been created")


The collection schema has been created


### **Add data/vectors to db**

- Turn dataframe into correct json format
- Loop through json, vectorize, and add vectors to weaviate

In [14]:
#new Issy's file
df_original = pd.read_csv("news-articles-results.csv")
df_original.head()

,SUBJECT,SOURCE_ID,NAME,AUTHOR,TITLE,URL,URL_TO_IMAGE,PUBLISHED_AT,CONTENT_FULL
0,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,McDonald's sales dented by Israel-Gaza boycotts,https://www.bbc.co.uk/news/business-68209085,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-02-05T17:38:09Z,"McDonald's has missed a key sales target, part..."
1,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,"Hamas command in north Gaza destroyed, Israel ...",https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-06T20:36:02Z,The Israeli army says it has 'completed the di...
2,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,South Africa's genocide case against Israel: B...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-12T20:59:40Z,The UN's top legal body has now heard two days...
3,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,UN court to hear South Africa genocide case ag...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-11T07:16:13Z,This video can not be played\n\nWatch: 'Israel...
4,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,Khan Younis: Israel says forces have encircled...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-23T21:15:25Z,The Israeli military says its ground forces ha...


In [15]:
df_original.columns

Index(['SUBJECT', 'SOURCE_ID', 'NAME', 'AUTHOR', 'TITLE', 'URL',
       'URL_TO_IMAGE', 'PUBLISHED_AT', 'CONTENT_FULL'],
      dtype='object')

In [5]:
## load dataset Issy

dfx = pd.read_csv("news-articles-results.csv")
dfx.columns = dfx.columns.str.lower()
dfx.head()

,subject,source_id,name,author,title,url,url_to_image,published_at,content_full
0,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,McDonald's sales dented by Israel-Gaza boycotts,https://www.bbc.co.uk/news/business-68209085,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-02-05T17:38:09Z,"McDonald's has missed a key sales target, part..."
1,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,"Hamas command in north Gaza destroyed, Israel ...",https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-06T20:36:02Z,The Israeli army says it has 'completed the di...
2,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,South Africa's genocide case against Israel: B...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-12T20:59:40Z,The UN's top legal body has now heard two days...
3,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,UN court to hear South Africa genocide case ag...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-11T07:16:13Z,This video can not be played\n\nWatch: 'Israel...
4,Israel,bbc-news,BBC News,https://www.facebook.com/bbcnews,Khan Younis: Israel says forces have encircled...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-01-23T21:15:25Z,The Israeli military says its ground forces ha...


In [22]:
dfx['name'].unique()

array(['BBC News', 'NPR', 'Business Insider', 'Engadget', 'Time',
       'The Atlantic', 'Gizmodo.com', 'ABC News', 'HuffPost', 'ReadWrite',
       'Wired', 'Yahoo Entertainment', 'MarketWatch', 'kottke.org',
       'Boing Boing', 'Pitchfork', 'Uscis.gov', 'Nikkei.com',
       'Theonion.com', 'Politico', 'Associated Press', 'Awais.io',
       'Rolling Stone', 'Al Jazeera English', 'VentureBeat',
       'Ars Technica', 'Asteriskmag.com', 'Openai.com', 'Windows Central',
       'Theregister.com', 'Vox', 'Thedefensepost.com', 'The Verge',
       'CNET', 'ESPN', 'Digital Trends', 'Slate Magazine', 'Kotaku',
       'Deadspin', '@tucson_police', 'Bleacher Report', 'PetaPixel',
       'Afterbabel.com', 'Slashdot.org', 'Phys.Org',
       'The Jewish Press - JewishPress.com', 'Science Daily',
       'Westpoint.edu', 'Techdirt', 'The Intercept', 'Skift', 'CNN',
       'Substack.com', 'Biztoc.com', 'The Indian Express', 'MacRumors',
       'Polygraph.info', 'Vice News', 'The New Yorker'], dtype=o

In [6]:
#Read the excel (to save on number of request used for News API)
#df_original = pd.read_excel("news_2024_02_02.xlsx")

#change df to json
result = dfx.to_json(orient="records")
parsed = loads(result)

In [7]:
parsed[0]

{'subject': 'Israel',
 'source_id': 'bbc-news',
 'name': 'BBC News',
 'author': 'https://www.facebook.com/bbcnews',
 'title': "McDonald's sales dented by Israel-Gaza boycotts",
 'url': 'https://www.bbc.co.uk/news/business-68209085',
 'url_to_image': 'https://ichef.bbci.co.uk/news/1024/branded_news/14FF/production/_132557350_gettyimages-1259083300.jpg',
 'published_at': '2024-02-05T17:38:09Z',
 'content_full': "McDonald's has missed a key sales target, partly due to customers boycotting the firm for its perceived support of Israel.\n\nThe fast food chain reported its first quarterly sales miss in nearly four years due to weak growth in its international business division.\n\nIts boss previously acknowledged the impact of the conflict, blaming 'misinformation'.\n\nShares in McDonald's fell about 4% after the announcement.\n\nMcDonalds is one of several Western corporations including Starbucks and Coca Cola that have seen boycotts and protests against them by anti-Israeli campaigners.\n\n

In [13]:
## Loop through json and add to weaviate
client.batch.configure(batch_size=100) 
with client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(parsed):  # Batch import data
        print(f"importing row: {i+1}")
        properties = {
                        "subject": d["subject"],
                        "source_id": d["source_id"],
                        "name": d["name"],
                        "author": d["author"],
                        "title": d["title"],
                        "url": d["url"],
                        "url_to_image": d["url_to_image"],
                        "published_at": d["published_at"],
                        "content_full": d["content_full"]}
        
                    
        batch.add_data_object(data_object=properties, class_name="news_articles")

importing row: 1
importing row: 2
importing row: 3
importing row: 4
importing row: 5
importing row: 6
importing row: 7
importing row: 8
importing row: 9
importing row: 10
importing row: 11
importing row: 12
importing row: 13
importing row: 14
importing row: 15
importing row: 16
importing row: 17
importing row: 18
importing row: 19
importing row: 20
importing row: 21
importing row: 22
importing row: 23
importing row: 24
importing row: 25
importing row: 26
importing row: 27
importing row: 28
importing row: 29
importing row: 30
importing row: 31
importing row: 32
importing row: 33
importing row: 34
importing row: 35
importing row: 36
importing row: 37
importing row: 38
importing row: 39
importing row: 40
importing row: 41
importing row: 42
importing row: 43
importing row: 44
importing row: 45
importing row: 46
importing row: 47
importing row: 48
importing row: 49
importing row: 50
importing row: 51
importing row: 52
importing row: 53
importing row: 54
importing row: 55
importing row: 56
i

### **Check number of records in weaviate**

In [14]:
# Test that all data has loaded – get object count
result = (
    client.query.aggregate("news_articles")
    .with_meta_count()
    .do()
)

result

{'data': {'Aggregate': {'News_articles': [{'meta': {'count': 1000}}]}}}

### **Example search with filter**

In [16]:
response = (
    client.query
    .get("news_articles", ['subject', 'source_id', 'name', 'author', 'title', 'url', 'urlToImage', 'publishedAt', 'content_full'])
    .with_near_text({"concepts": ["Why did Biden win the elections?"]})
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "News_articles": [
                {
                    "author": "Benjamin Wallace-Wells",
                    "content_full": "To revisit this article, select My Account, thenView saved stories\n\nFind anything you save across the site in your account\n\nBy Benjamin Wallace-Wells\n\nEverything about Joe Bidens relection effort tends to be pretty quiet, and so it was with a characteristic mutedness that the campaign greeted its best week in a long time. Much of the good news had to do with the economy, as the stock market broke records and prices have come under control. The war on inflation is over. We won, Paul Krugman, the Nobel economics laureate, had declared. Donald Trump has been attacking the economys performance since his reluctant departure from office, and so, when he tried to take credit for it last week, it seemed a good gauge of how much the wind is shifting. this is the trump stock market, he wrote on Truth Social, because m

## **Search with Cohere prompt**

In [19]:

context = "to an adult"  #not used yet
name_of_source = ["Time", "BBC News", "Business Insider"] 

response = (client.query
            .get("news_articles", ['subject', 'source_id', 'name', 'author', 'title', 'url', 'urlToImage', 'publishedAt', 'content_full'])
            .with_near_text({"concepts": ["Why did Biden win the elections?"]})
            .with_generate(single_prompt="Explain {content_full} as you might to an adult")                   #uses Cohere command to generate respons
            #.with_additional("rerank(property: \"answer\" query: \"content\") { score }")                #reranking using cohere
#             .with_where({
#                   "path": ['name'],                                                                       # Search in 'name' which equals 'Time'
#                   "operator": "Equal", 
#                   "valueText": 'Time'
#             })
            .with_limit(5).do())
    
response

{'data': {'Get': {'News_articles': [{'_additional': {'generate': {'error': None,
       'singleResult': "The New Yorker offers a variety of articles on a range of topics, from political analysis to cultural commentary and personal essays. In the issue highlighted, there are articles focusing on the upcoming US election, with an analysis of the current political climate and the strategies of the major candidates. There are also articles on personal experiences, such as learning to cycle in the Netherlands and the impact of a terrible crime on an individual's life. The issue also features a Shouts & Murmurs piece on the challenges of being an introvert, as well as a reflection on the legacy of Julia Child and her impact on cooking. These articles offer readers a diverse range of perspectives and insights, providing an intellectually stimulating and entertaining reading experience."}},
     'author': 'Benjamin Wallace-Wells',
     'content_full': 'To revisit this article, select My Accoun

In [ ]:
# Source URLs

In [25]:
import gradio as gr


def generate_bot_response(chatbot, input_question, list_of_sources):
    
    output_text = "Hello " + input_question
    
    
    return chatbot


with gr.Blocks() as demo:

    with gr.Row():
            with gr.Row():
              # Chatbot interface
              chatbot = gr.Chatbot(label="Cohere LLM",
                                   value=[],
                                   elem_id='chatbot',
                                   render=True,
                                    bubble_full_width=False)
                
                
            with gr.Column():

                list_of_sources = gr.CheckboxGroup(['BBC News', 'NPR', 'Business Insider', 'Engadget', 'Time',
                                                       'The Atlantic', 'Gizmodo.com', 'ABC News', 'HuffPost', 'ReadWrite',
                                                       'Wired', 'Yahoo Entertainment', 'MarketWatch', 'kottke.org',
                                                       'Boing Boing', 'Pitchfork', 'Uscis.gov', 'Nikkei.com',
                                                       'Theonion.com', 'Politico', 'Associated Press', 'Awais.io',
                                                       'Rolling Stone', 'Al Jazeera English', 'VentureBeat',
                                                       'Ars Technica', 'Asteriskmag.com', 'Openai.com', 'Windows Central',
                                                       'Theregister.com', 'Vox', 'Thedefensepost.com', 'The Verge',
                                                       'CNET', 'ESPN', 'Digital Trends', 'Slate Magazine', 'Kotaku',
                                                       'Deadspin', '@tucson_police', 'Bleacher Report', 'PetaPixel',
                                                       'Afterbabel.com', 'Slashdot.org', 'Phys.Org',
                                                       'The Jewish Press - JewishPress.com', 'Science Daily',
                                                       'Westpoint.edu', 'Techdirt', 'The Intercept', 'Skift', 'CNN',
                                                       'Substack.com', 'Biztoc.com', 'The Indian Express', 'MacRumors',
                                                       'Polygraph.info', 'Vice News', 'The New Yorker'], label="Sources", info="Which media would you like to include?"),

            
    with gr.Column():
        with gr.Column():
            input_question = gr.Text(show_label=False, placeholder="Enter question")

        with gr.Column():
          # button to submit question to the bot
          submit_btn = gr.Button('Ask your question here!')

    submit_btn.click(
          fn=generate_bot_response,
          inputs=[chatbot, input_question, list_of_sources],
          outputs=[chatbot])


if __name__ == "__main__":
#     demo.queue()
    demo.launch(share=True, debug=True) # launch app

AttributeError: 'tuple' object has no attribute '_id'